# 언어 모델의 기초적 이해

### 데이터 다운로드 받고 살펴보기

`wikipedia-api`는 인터넷에서 위키피디아 문서를 불러올 수 있도록 도와줍니다. `google colab` 환경에서 실행할 경우, 아래 코드를 실행해 라이브러리를 다운받아봅시다.

In [ ]:
!pip install wikipedia-api

In [ ]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Breakfast")
text = p_wiki.text
print(text)

### 데이터 전처리

인터넷에서 텍스트를 그대로 다운받아 사용하면 대부분의 경우 텍스트 데이터가 잘 정돈되어 있지 않습니다.
따라서 불필요한 텍스트를 제거하고, 학습에 용이한 형태로 텍스트를 수정해야합니다.

In [ ]:
import re

def split_text_to_sentences(text):
    # A basic sentence tokenizer
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

def remove_text_from_start_end_marker(text, start_marker='(', end_marker=')'):
    # Remove parentheses and their content
    return re.sub(r'\{}.*?\{}'.format(re.escape(start_marker), re.escape(end_marker)), '', text).strip()

def clean_text_data(text):

    sentences = split_text_to_sentences(text)
    sentences = [i.lower() for i in sentences] # make sentence lower cased. e.g. "Hello World" -> "hello world"
    sentences = [remove_text_from_start_end_marker(i) for i in sentences] # remove parentheses and their content. e.g. "hello world (test)" -> "hello world"

    # Some sentences are just too long.
    # We will split them into smaller sentences.
    short_sentences = []
    for i in sentences:
        temp = i.split(',')
        for j in temp:
            short_sentences.append(j.strip())

    # Remove undesirable characters
    to_replace = ["!", ";", '\n', '</p>', '<a', 'id=', "href=", 'title=', 'class=', '</a>', '(', ')', '}', '{',
                  '</sup>', '<p>', '</b>', '<sup', '>', '<', '\\', '-']
    replace_with = ''

    cleaned_sentences = []
    for i in short_sentences:
        word_array = i.split()
        word_array_new = []
        for word in word_array:
            for to_replace_val in to_replace:
                word = word.replace(to_replace_val, replace_with)
            word_array_new.append(word)
        cleaned_sentence = ' '.join(word_array_new).strip()
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence) # Remove extra whitespaces
        cleaned_sentences.append(cleaned_sentence)

    # Now some sentences are too short.
    # We will remove them.
    cleaned_sentences = [i for i in cleaned_sentences if len(i.split()) > 10]
    return cleaned_sentences

In [ ]:
sentences = clean_text_data(text)
print(sentences)

### 다음 단어가 등장할 확률 계산

이제 우리는 언어 모델의 기초를 살펴봅니다. 
언어 모델은 지금까지 생성된 텍스트를 바탕으로 다음에 등장할 단어(토큰)의 확률을 토대로 새로운 단어를 생성합니다.  

예를 들어, 가장 단순한 형태의 언어모델은, 이전 단어 다음에 등장할 가장 높은 확률의 단어를 텍스트에서 구해서 그 단어를 다음 단어로 채택합니다. 

먼저, 각 단어의 다음으로 등장하는 단어를 전부 세어서, 다음에 등장할 단어의 확률을 계산해봅시다.

In [ ]:
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Load Hugging Face tokenizer (you can change the model name!)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_next_token_probabilities(sentences, given_token_text, tokenizer=None):
    # Check if tokenizer is provided
    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    # Tokenize all sentences into token ids
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)

    # Convert the given token text to token id
    given_token_id = tokenizer.convert_tokens_to_ids(given_token_text)

    # Dictionary to store next-token counts
    next_token_counts = defaultdict(Counter)

    # Populate next-token counts
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1

    # Calculate probabilities
    total_next = sum(next_token_counts[given_token_id].values())
    if total_next == 0:
        return {}

    probabilities = {
        tokenizer.convert_ids_to_tokens(token_id): count / total_next
        for token_id, count in next_token_counts[given_token_id].items()
    }
    return probabilities

In [ ]:
given_token_text = 'breakfast'
probabilities = compute_next_token_probabilities(sentences, given_token_text)

# Output probabilities
for next_token, prob in sorted(probabilities.items(), key=lambda x: x[1], reverse=True):
    print(f"'{given_token_text}' → '{next_token}': {prob:.2f}")

### 문장 생성하기

이제 우리는 기초적인 언어 모델을 완성했습니다.  

매번 단어를 생성할 때마다, 이전 단어 다음에 가장 높은 확률로 등장할 단어를 생성하도록 선택해서 문장을 이어나가 봅시다.

In [ ]:
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_next_token_counts(tokens):
    next_token_counts = defaultdict(Counter)
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1
    return next_token_counts

def prepare_token_data(sentences, tokenizer=None):
    # Tokenize all sentences into token ids
    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)
    return tokens

def greedy_generate_sentence(sentences, start_token_text, tokenizer=None, max_length=20):
    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = prepare_token_data(sentences, tokenizer=tokenizer)
    next_token_counts = compute_next_token_counts(tokens)

    # Start token
    current_token_id = tokenizer.convert_tokens_to_ids(start_token_text)
    if current_token_id == tokenizer.unk_token_id:
        print(f"Warning: '{start_token_text}' is unknown in the tokenizer vocabulary.")

    generated_tokens = [current_token_id]

    for _ in range(max_length):
        # Get next token counts
        next_counts = next_token_counts.get(current_token_id, None)
        if not next_counts:
            break  # No next token found

        # Greedily pick the most probable next token
        next_token_id = next_counts.most_common(1)[0][0]
        generated_tokens.append(next_token_id)

        # Update current token
        current_token_id = next_token_id

        # Optional: break if punctuation token or special token is reached
        token_text = tokenizer.convert_ids_to_tokens(current_token_id)
        if token_text in ['.', '!', '?', tokenizer.sep_token, tokenizer.pad_token]:
            break

    # Convert token ids back to text
    generated_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(generated_tokens))
    return generated_text


In [ ]:
# Example usage:
start_token_text = 'breakfast'
generated_sentence = greedy_generate_sentence(sentences, start_token_text)
print("Generated sentence:")
print(generated_sentence)

### 문제점 이해하기

문장을 생성하고 보니, 여러가지 문제점이 보입니다:

1. 항상 같은 문장만 생성됩니다. 다음으로 등장할 확률이 가장 높은 단어만 탐욕적으로 선택하기 때문에, 문장이 바뀌지 않습니다.
2. 무한히 반복되는 문장이 생성될 때도 있습니다. 예를 들어, `너를` 다음에 `사랑하지만`가 가장 빈번하게 등장하고, 이어서 `사랑하지만` 다음에는 다시 `너를`이 등장할 확률이 가장 높다면, 문장은 `너를 사랑하지만 너를 사랑하지만...` 처럼 무한히 반복됩니다.

이걸 해결할 수 있는 방법은 무엇일까요? 

두 가지 문제점을 동시에 해결할 수 있는 가장 간단한 방법은, 다음으로 등장할 가장 높은 확률의 단어만 선택하는 것이 아니라, 조금 확률이 낮은 단어라도 생성될 수 있도록 확률을 토대로 랜덤하게 고르는 것입니다. 한번 해결해볼까요?

In [ ]:
import random


def sample_next_token(next_counts):
    tokens, counts = zip(*next_counts.items())
    total = sum(counts)
    probabilities = [count / total for count in counts]
    return random.choices(tokens, weights=probabilities, k=1)[0]

def random_sample_generate_sentence(sentences, start_token_text, tokenizer=None, max_length=20):
    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = prepare_token_data(sentences, tokenizer=tokenizer)
    next_token_counts = compute_next_token_counts(tokens)

    # Start token
    current_token_id = tokenizer.convert_tokens_to_ids(start_token_text)
    if current_token_id == tokenizer.unk_token_id:
        print(f"Warning: '{start_token_text}' is unknown in the tokenizer vocabulary.")

    generated_tokens = [current_token_id]

    for _ in range(max_length):
        # Get next token counts
        next_counts = next_token_counts.get(current_token_id, None)
        if not next_counts:
            break  # No next token found

        # Sample next token from distribution
        next_token_id = sample_next_token(next_counts)
        generated_tokens.append(next_token_id)

        # Update current token
        current_token_id = next_token_id

        # Optional: stop if punctuation token or special token
        token_text = tokenizer.convert_ids_to_tokens(current_token_id)
        if token_text in ['.', '!', '?', tokenizer.sep_token, tokenizer.pad_token]:
            break

    # Convert token ids back to text
    generated_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(generated_tokens))
    return generated_text


In [ ]:
# Example usage:
start_token_text = 'breakfast'
generated_sentence = random_sample_generate_sentence(sentences, start_token_text)
print("Generated sentence (random sample):")
print(generated_sentence)

### 더 발전할 방향 논의해보기

그래도 몇 가지 문제점이 보입니다. 한 번 같이 논의해보고, 앞으로 어떻게 수정할 수 있을지 고민해봅시다.